In [1]:
# Data Loading 
import numpy as np
import pandas as pd
from tqdm import tqdm

base_path = './MLEnd/deception/MLEndDD_stories_small/'
MLEND_df = pd.read_csv('./MLEnd/deception/MLEndDD_story_attributes_small.csv').set_index('filename')

files = [base_path + file for file in MLEND_df.index]

print(f"We have {len(files)} audio files in the dataset.")
display(MLEND_df.head())

#Langauge and Data Distribution
language_counts = MLEND_df['Language'].value_counts()
language_df = pd.DataFrame(language_counts).transpose()
language_df['Sum'] = language_counts.sum()

story_type_counts = MLEND_df['Story_type'].value_counts()

print("Languages narrated in the dataset are:")
display(language_df)
print(story_type_counts)

We have 100 audio files in the dataset.


,Language,Story_type
filename,,
00001.wav,Hindi,deceptive_story
00002.wav,English,true_story
00003.wav,English,deceptive_story
00004.wav,Bengali,deceptive_story
00005.wav,English,deceptive_story


Languages narrated in the dataset are:


Language,English,Hindi,Arabic,"Chinese, Mandarin",Marathi,Bengali,Kannada,French,Russian,Portuguese,Spanish,Swahilli,Telugu,Korean,Cantonese,Italian,Sum
count,78,4,3,2,2,1,1,1,1,1,1,1,1,1,1,1,100


Story_type
deceptive_story    50
true_story         50
Name: count, dtype: int64


In [2]:
import librosa

def split_audio(file_id, file_path, label, chunk_duration=30, sr=None):
    """
    Splits an audio file into fixed-length chunks, analyzes valid and non-valid chunks,
    and calculates data loss during splitting.

    Args:
        file_id (str): The name of the audio file (e.g., '00001.wav').
        file_path (str): Path to the audio file.
        label (str): The label for the audio file (e.g., 'true_story' or 'deceptive_story').
        chunk_duration (int): Duration of each chunk in seconds (default is 30).
        sr (int or None): Sampling rate. If None, the original rate is used.

    Returns:
        dict: Metadata about the file, including total duration, valid and non-valid chunk counts, and data loss.
        list: Information about each valid chunk, including its ID and label.
    """
    audio_data, sample_rate = librosa.load(file_path, sr=sr)  # sr=None uses original sample rate
    
    total_duration = len(audio_data) / sample_rate #in seconds
    chunk_size = int(chunk_duration * sample_rate)

    # Split the audio into chunks of `chunk_size`
    chunks = [audio_data[i:i + chunk_size] for i in range(0, len(audio_data), chunk_size)]
    
    valid_chunks = [chunk for chunk in chunks if len(chunk) == chunk_size]
    non_valid_chunks = [chunk for chunk in chunks if len(chunk) < chunk_size]

    metadata = {
        "File ID": file_id,
        "Duration (s)": total_duration,
        "Sample Rate": sample_rate,
        "Total Chunks": len(chunks),
        "Valid Chunks (30s)": len(valid_chunks),
        "Non-Valid Chunks (<30s)": len(non_valid_chunks),
        "Label": label
    }

    chunk_info = [
        {"File ID": file_id, "Chunk ID": f"{file_id}_chunk{i + 1}", "Chunk Data": chunk, "Label": label, "Sample Rate": sample_rate}
        for i, chunk in enumerate(valid_chunks)
    ]

    return metadata, chunk_info


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# Split audio into 30s
file_metadata = []
audio_chunks = []

for file_id in tqdm(MLEND_df.index):  
    file_path = base_path + file_id   
    label = MLEND_df.loc[file_id, 'Story_type'] 

    metadata, chunks = split_audio(file_id, file_path, label)
    file_metadata.append(metadata) 
    audio_chunks.extend(chunks)     

# Metadata DF
metadata_df = pd.DataFrame(file_metadata)
print("Summary of Audio Files:")
display(metadata_df.head())

# Chunk Info DF
chunks_df = pd.DataFrame(audio_chunks)
print("Summary of Valid Audio Chunks:")
display(chunks_df[["File ID", "Chunk ID", "Label"]].head())


# Summary statistics
print("\nSummary Statistics:")
print(f"Total Files Processed: {len(metadata_df)}")
print(f"Total Chunks Created: {metadata_df['Total Chunks'].sum()}")
print(f"Total Valid Chunks(30s): {metadata_df['Valid Chunks (30s)'].sum()}")
print(f"Total Non-Valid Chunks (<30s): {metadata_df['Non-Valid Chunks (<30s)'].sum()}")

# True and Deceptive Distribution after splitting
valid_chunk_labels = chunks_df['Label'].value_counts()
print("\nCount of True and Deceptive Stories from Valid Chunks (30s):")
for label, count in valid_chunk_labels.items():
    print(f"{label}: {count} chunks")

100%|██████████| 100/100 [00:06<00:00, 16.61it/s]

Summary of Audio Files:


,File ID,Duration (s),Sample Rate,Total Chunks,Valid Chunks (30s),Non-Valid Chunks (<30s),Label
0,00001.wav,122.167256,44100,5,4,1,deceptive_story
1,00002.wav,125.192018,44100,5,4,1,true_story
2,00003.wav,162.984127,44100,6,5,1,deceptive_story
3,00004.wav,121.681270,44100,5,4,1,deceptive_story
4,00005.wav,134.189751,44100,5,4,1,deceptive_story


Summary of Valid Audio Chunks:


,File ID,Chunk ID,Label
0,00001.wav,00001.wav_chunk1,deceptive_story
1,00001.wav,00001.wav_chunk2,deceptive_story
2,00001.wav,00001.wav_chunk3,deceptive_story
3,00001.wav,00001.wav_chunk4,deceptive_story
4,00002.wav,00002.wav_chunk1,true_story



Summary Statistics:
Total Files Processed: 100
Total Chunks Created: 520
Total Valid Chunks(30s): 420
Total Non-Valid Chunks (<30s): 100

Count of True and Deceptive Stories from Valid Chunks (30s):
true_story: 219 chunks
deceptive_story: 201 chunks


Dataset

A total of 100 audio recordings - 50 true and 50 deceptive 
16 languages - English is the most dominant, followed by Hindi, Arabic, Chinese (Mandarin), Marathi and other languages as shown in the code above. 

After splitted the audio into 30s chunks and discard those less than 30s chunks to remain consistency. As a result,we have remaning of 420 valid 30s chunks, and 219 of them are true, 201 are false. This results in the following proportions:

True Stories: 
219/420 ≈52.14%

Deceptive Stories: 
201/420 ≈47.86%

Hence, this dataset is generally considered balanced as both classes (true and deceptive) are roughly equal.

### Feature extraction

In [4]:
import numpy as np
import librosa
from tqdm import tqdm

def extract_features_and_labels(chunks_df, scale_audio=True):
    """
    Extract MFCC, Pitch, Energy, and ZCR features from audio chunks and associate labels.
    Includes File IDs to enable group-based splitting and avoid data leakage.

    Args:
        chunks_df (DataFrame): DataFrame containing Chunk Data, Label, Sample Rate, and File ID.
        scale_audio (bool): Whether to scale audio amplitude.

    Returns:
        dict: Feature matrices for MFCC, Pitch, Energy, ZCR.
        np.ndarray: Labels for each chunk.
        list: File IDs for grouping to avoid data leakage.
    """
    # Initialize 
    file_ids, mfcc_features, pitch_features, energy_features, zcr_features, labels = [], [], [], [], [], []

    for index, row in tqdm(chunks_df.iterrows(), total=len(chunks_df)):
        audio_data, label, sr, file_id = row["Chunk Data"], row["Label"], row["Sample Rate"], row["File ID"]

        if scale_audio:
            audio_data = audio_data / np.max(np.abs(audio_data))

        # Feature 1: MFCC
        mfcc = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13).mean(axis=1)
        mfcc_features.append(mfcc)

        # Feature 2: Pitch
        pitch, _, _ = librosa.pyin(audio_data, fmin=80, fmax=450, sr=sr)
        pitch_mean = np.nanmean(pitch) if np.mean(np.isnan(pitch))<1 else 0
        pitch_std  = np.nanstd(pitch) if np.mean(np.isnan(pitch))<1 else 0
        pitch_features.append([pitch_mean, pitch_std])

        # Feature 3: Energy
        rms = np.mean(librosa.feature.rms(y=audio_data))
        energy_features.append([rms])

        # Feature 4: Zero-Crossing Rate (ZCR)
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=audio_data))
        zcr_features.append([zcr])

        labels.append(1 if label == 'deceptive_story' else 0)
        file_ids.append(file_id)

    return {
        'MFCC': np.array(mfcc_features),
        'Pitch': np.array(pitch_features),
        'Energy': np.array(energy_features),
        'ZCR': np.array(zcr_features)
    }, np.array(labels), file_ids


In [ ]:
# Extract features, labels, and File IDs
features, labels, file_ids = extract_features_and_labels(chunks_df, scale_audio=True) 

100%|██████████| 420/420 [14:14<00:00,  2.03s/it]


In [22]:
X_mfcc = features['MFCC']
X_pitch = features['Pitch']
X_energy = features['Energy']
X_zcr = features['ZCR']
X_combined = np.hstack((X_mfcc, X_pitch, X_energy, X_zcr))
y = labels

print(f"MFCC Shape: {X_mfcc.shape}")
print(f"Pitch Shape: {X_pitch.shape}")
print(f"Energy Shape: {X_energy.shape}")
print(f"ZCR Shape: {X_zcr.shape}")
print(f"Combined Feature Matrix Shape: {X_combined.shape}")
print(f"Labels (y) Shape: {y.shape}")
print(f"File IDs Length: {len(file_ids)}")


MFCC Shape: (420, 13)
Pitch Shape: (420, 2)
Energy Shape: (420, 1)
ZCR Shape: (420, 1)
Combined Feature Matrix Shape: (420, 17)
Labels (y) Shape: (420,)
File IDs Length: 420


Train test split

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
import numpy as np

def group_train_valid_split_and_scale(features_dict, y, groups, valid_size=0.3, random_state=42):
    """
    Perform group-aware train-validation split and standardize features.

    Args:
        features_dict (dict): Dictionary of feature matrices (e.g., {"MFCC": X_mfcc, "Pitch": X_pitch}).
        y (np.ndarray): Labels.
        groups (list or np.ndarray): Group IDs (e.g., File IDs) for each sample.
        valid_size (float): Proportion of the data to be used as the validation set.
        random_state (int): Random seed for reproducibility.

    Returns:
        tuple: Scaled train and validation feature dictionaries, train and validation labels, and indices:
               (scaled_train_features, scaled_valid_features, y_train, y_valid, train_idx, valid_idx).
    """
    # Group-aware splitting
    gss = GroupShuffleSplit(n_splits=1, test_size=valid_size, random_state=random_state)
    train_idx, valid_idx = next(gss.split(list(features_dict.values())[0], y, groups))

    # Initialize dictionaries for scaled features
    scaled_train_features = {}
    scaled_valid_features = {}

    # Split 
    for feature_name, feature_matrix in features_dict.items():
        X_train, X_valid = feature_matrix[train_idx], feature_matrix[valid_idx]
        
        # Standardize
        scaler = StandardScaler()
        scaled_train_features[feature_name] = scaler.fit_transform(X_train)
        scaled_valid_features[feature_name] = scaler.transform(X_valid)

    # Split labels
    y_train, y_valid = y[train_idx], y[valid_idx]

    return scaled_train_features, scaled_valid_features, y_train, y_valid, train_idx, valid_idx


features_dict = {"MFCC": X_mfcc, "Pitch": X_pitch, "Energy": X_energy, "ZCR": X_zcr, "Combined": X_combined}

scaled_train_features, scaled_valid_features, y_train, y_valid, train_idx, valid_idx = group_train_valid_split_and_scale(
    features_dict, y, file_ids, valid_size=0.3, random_state=42)

print("Train-Validation Split:")
for feature_name in scaled_train_features.keys():
    print(f"{feature_name} - Train: {scaled_train_features[feature_name].shape}, Validation: {scaled_valid_features[feature_name].shape}")
print(f"Labels - Train: {len(y_train)}, Validation: {len(y_valid)}")


Train-Validation Split:
MFCC - Train: (287, 13), Validation: (133, 13)
Pitch - Train: (287, 2), Validation: (133, 2)
Energy - Train: (287, 1), Validation: (133, 1)
ZCR - Train: (287, 1), Validation: (133, 1)
Combined - Train: (287, 17), Validation: (133, 17)
Labels - Train: 287, Validation: 133


Models

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

models = {
    "LogisticRegression": LogisticRegression(),
    "RandomForest": RandomForestClassifier(max_depth=5),
    "GradientBoosting": GradientBoostingClassifier(),
    "SVM": SVC(C=1, gamma='scale', probability=True)
}

# Function to train and evaluate models
def train_and_evaluate_model(model, X_train, X_valid, y_train, y_valid):
    """
    Train a model and evaluate its performance on the training and validation sets.

    Args:
        model: A scikit-learn classifier.
        X_train, X_valid: Scaled training and validation features.
        y_train, y_valid: Corresponding labels.

    Returns:
        dict: Training and validation accuracy and F1 scores.
    """
    
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)
    
    metrics = {
        "Training Accuracy": accuracy_score(y_train, y_train_pred),
        "Validation Accuracy": accuracy_score(y_valid, y_valid_pred),
        "Training F1 Score": f1_score(y_train, y_train_pred),
        "Validation F1 Score": f1_score(y_valid, y_valid_pred),
    }
    return metrics

results = []

for feature_name, X_train in scaled_train_features.items():
    # if feature_name == "Combined":
    #     continue  # Skip the "Combined" features

    X_valid = scaled_valid_features[feature_name]
    y_train_split, y_valid_split = y_train, y_valid

    
    for model_name, model in models.items():
        metrics = train_and_evaluate_model(model, X_train, X_valid, y_train_split, y_valid_split)
        
        results.append({"Feature": feature_name,
                         "Model": model_name,
                         "Training Accuracy": metrics["Training Accuracy"],
                         "Validation Accuracy": metrics["Validation Accuracy"],
                         "Training F1 Score": metrics["Training F1 Score"],
                         "Validation F1 Score": metrics["Validation F1 Score"]})

results_df = pd.DataFrame(results)
print("Training Results Summary:")
display(results_df)



Training Results Summary:


,Feature,Model,Training Accuracy,Validation Accuracy,Training F1 Score,Validation F1 Score
0,MFCC,LogisticRegression,0.634146,0.518797,0.553191,0.360000
1,MFCC,RandomForest,0.982578,0.548872,0.979592,0.387755
2,MFCC,GradientBoosting,0.996516,0.533835,0.995951,0.483333
3,MFCC,SVM,0.975610,0.556391,0.971429,0.486957
4,Pitch,LogisticRegression,0.567944,0.421053,0.000000,0.000000
5,Pitch,RandomForest,0.808362,0.511278,0.789272,0.585987
6,Pitch,GradientBoosting,0.940767,0.533835,0.932271,0.569444
7,Pitch,SVM,0.599303,0.518797,0.397906,0.457627
8,Energy,LogisticRegression,0.585366,0.436090,0.278788,0.242424
9,Energy,RandomForest,0.766551,0.413534,0.663317,0.350000


Ensemble

In [107]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score

# Ensemble models
def train_ensemble_voting(X_train, X_valid, y_train, y_valid, models_dict):
    """
    Train an ensemble voting classifier and evaluate it on training and validation sets.

    Args:
        X_train (np.ndarray): Training features.
        X_valid (np.ndarray): Validation features.
        y_train (np.ndarray): Training labels.
        y_valid (np.ndarray): Validation labels.
        models_dict (dict): Dictionary of models to include in the ensemble.

    Returns:
        dict: Training and validation accuracy and F1 scores.
    """
    # Create a VotingClassifier with soft voting
    ensemble = VotingClassifier(estimators=list(models_dict.items()), voting='soft')
    ensemble.fit(X_train, y_train)

    # Predict on training and validation sets
    y_train_pred = ensemble.predict(X_train)
    y_valid_pred = ensemble.predict(X_valid)

    # Calculate metrics
    metrics = {
        "Training Accuracy": accuracy_score(y_train, y_train_pred),
        "Validation Accuracy": accuracy_score(y_valid, y_valid_pred),
        "Training F1 Score": f1_score(y_train, y_train_pred),
        "Validation F1 Score": f1_score(y_valid, y_valid_pred),
    }

    return ensemble, metrics


# Initialize results list for ensembles
ensemble_results = []

# Train and evaluate ensemble models for each feature set (excluding "Combined")
for feature_name, X_train in scaled_train_features.items():
    # if feature_name == "Combined":
    #     continue  # Skip the "Combined" features

    X_valid = scaled_valid_features[feature_name]
    y_train_split, y_valid_split = y_train, y_valid

    # Train ensemble using individual models for this feature
    ensemble_model, metrics = train_ensemble_voting(
        X_train, X_valid, y_train_split, y_valid_split, models
    )

    # Store results
    ensemble_results.append({
        "Feature": feature_name,
        **metrics
    })

# Convert ensemble results to a DataFrame
ensemble_results_df = pd.DataFrame(ensemble_results)

# Display the DataFrame
print("\nEnsemble Results Summary:")
display(ensemble_results_df)



Ensemble Results Summary:


,Feature,Training Accuracy,Validation Accuracy,Training F1 Score,Validation F1 Score
0,MFCC,0.993031,0.466165,0.991935,0.348624
1,Pitch,0.888502,0.488722,0.863248,0.492537
2,Energy,0.770035,0.451128,0.645161,0.342342
3,ZCR,0.745645,0.368421,0.592179,0.106383
4,Combined,0.993031,0.473684,0.991935,0.285714


In [108]:
# Group results by feature and select the model with the highest validation F1 score
best_models = {}
for feature_name in results_df["Feature"].unique():
    feature_group = results_df[results_df["Feature"] == feature_name]
    best_model_row = feature_group.loc[feature_group["Validation F1 Score"].idxmax()]
    
    best_models[feature_name] = {
        "Model Name": best_model_row["Model"],
        "Validation F1 Score": best_model_row["Validation F1 Score"],
        "Validation Accuracy": best_model_row["Validation Accuracy"]
    }

# Display the selected models
print("Best Models for Each Feature:")
for feature, model_info in best_models.items():
    print(f"{feature}: {model_info['Model Name']} (Validation F1: {model_info['Validation F1 Score']}, Validation Accuracy: {model_info['Validation Accuracy']})")


Best Models for Each Feature:
MFCC: GradientBoosting (Validation F1: 0.4918032786885246, Validation Accuracy: 0.5338345864661654)
Pitch: GradientBoosting (Validation F1: 0.5694444444444444, Validation Accuracy: 0.5338345864661654)
Energy: GradientBoosting (Validation F1: 0.40625, Validation Accuracy: 0.42857142857142855)
ZCR: GradientBoosting (Validation F1: 0.30357142857142855, Validation Accuracy: 0.41353383458646614)
Combined: LogisticRegression (Validation F1: 0.36538461538461536, Validation Accuracy: 0.5037593984962406)


In [109]:
# Initialize the best models
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

best_models = {
    "MFCC": GradientBoostingClassifier(),
    "Pitch": RandomForestClassifier(),
    "Energy": GradientBoostingClassifier(),
    "ZCR": GradientBoostingClassifier()
}

# Train the selected best models on their respective features
trained_models = {}
for feature_name, model in best_models.items():
    X_train_feature = scaled_train_features[feature_name]
    model.fit(X_train_feature, y_train)
    trained_models[feature_name] = model


In [94]:
import numpy as np
from collections import Counter

def majority_voting(trained_models, scaled_valid_features):
    """
    Perform majority voting using predictions from trained models.
    
    Args:
        trained_models (dict): Trained models for each feature.
        scaled_valid_features (dict): Scaled validation features for each feature.
    
    Returns:
        np.ndarray: Final predictions using majority voting.
    """
    predictions = []
    for feature_name, model in trained_models.items():
        X_valid_feature = scaled_valid_features[feature_name]
        preds = model.predict(X_valid_feature)
        predictions.append(preds)
    
    # Transpose and vote
    predictions = np.array(predictions).T
    majority_votes = np.array([Counter(row).most_common(1)[0][0] for row in predictions])
    return majority_votes

# Get majority voting predictions on the validation set
y_valid_pred = majority_voting(trained_models, scaled_valid_features)


In [95]:
from sklearn.metrics import accuracy_score, f1_score

# Calculate validation metrics
inter_feature_metrics = {
    "Validation Accuracy": accuracy_score(y_valid, y_valid_pred),
    "Validation F1 Score": f1_score(y_valid, y_valid_pred)
}

# Display results
print("Inter-Feature Ensemble Validation Metrics:")
print(f"Validation Accuracy: {inter_feature_metrics['Validation Accuracy']:.4f}")
print(f"Validation F1 Score: {inter_feature_metrics['Validation F1 Score']:.4f}")


Inter-Feature Ensemble Validation Metrics:
Validation Accuracy: 0.4962
Validation F1 Score: 0.4553


In [105]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter

# Train a Baseline Model on Combined Features
def train_baseline_combined(X_train, X_valid, y_train, y_valid):
    """
    Train a baseline model on combined features and calculate validation metrics.
    
    Args:
        X_train (np.ndarray): Training features (combined).
        X_valid (np.ndarray): Validation features (combined).
        y_train (np.ndarray): Training labels.
        y_valid (np.ndarray): Validation labels.
    
    Returns:
        dict: Validation accuracy and F1 score for the baseline model.
    """
    # Define and train the baseline model (e.g., Gradient Boosting)
    baseline_model = SVC()
    baseline_model.fit(X_train, y_train)
    
    # Predict on the validation set
    y_valid_pred = baseline_model.predict(X_valid)
    
    # Calculate validation metrics
    metrics = {
        "Validation Accuracy": accuracy_score(y_valid, y_valid_pred),
        "Validation F1 Score": f1_score(y_valid, y_valid_pred)
    }
    return metrics, baseline_model


# Inter-Feature Ensemble Function
def inter_feature_ensemble(trained_models, scaled_valid_features, y_valid):
    """
    Perform inter-feature ensemble using majority voting on validation predictions.
    
    Args:
        trained_models (dict): Trained models for each feature.
        scaled_valid_features (dict): Scaled validation features for each feature.
        y_valid (np.ndarray): Validation labels.
    
    Returns:
        dict: Validation accuracy and F1 score for the inter-feature ensemble.
    """
    # Generate predictions for each feature model
    predictions_list = []
    for feature_name, model in trained_models.items():
        X_valid_feature = scaled_valid_features[feature_name]
        predictions = model.predict(X_valid_feature)
        predictions_list.append(predictions)
    
    # Perform majority voting
    predictions = np.array(predictions_list).T
    majority_votes = [Counter(row).most_common(1)[0][0] for row in predictions]
    
    # Calculate metrics
    metrics = {
        "Validation Accuracy": accuracy_score(y_valid, majority_votes),
        "Validation F1 Score": f1_score(y_valid, majority_votes)
    }
    return metrics


# Step 1: Train the Baseline Model on Combined Features
baseline_metrics, baseline_model = train_baseline_combined(
    X_combined_train, X_combined_valid, y_train, y_valid)

# Step 2: Train and Evaluate Inter-Feature Ensemble
# Train the best models on their respective features
trained_models = {}
for feature_name, model in best_models.items():
    X_train_feature = scaled_train_features[feature_name]
    model.fit(X_train_feature, y_train)
    trained_models[feature_name] = model

# Evaluate inter-feature ensemble
ensemble_metrics = inter_feature_ensemble(trained_models, scaled_valid_features, y_valid)

# Display the results
print("\nValidation Metrics:")
print(f"Baseline Model on Combined Features:")
print(f" - Validation Accuracy: {baseline_metrics['Validation Accuracy']:.4f}")
print(f" - Validation F1 Score: {baseline_metrics['Validation F1 Score']:.4f}")

print("\nInter-Feature Ensemble:")
print(f" - Validation Accuracy: {ensemble_metrics['Validation Accuracy']:.4f}")
print(f" - Validation F1 Score: {ensemble_metrics['Validation F1 Score']:.4f}")



Validation Metrics:
Baseline Model on Combined Features:
 - Validation Accuracy: 0.5263
 - Validation F1 Score: 0.3636

Inter-Feature Ensemble:
 - Validation Accuracy: 0.4812
 - Validation F1 Score: 0.4202


In [115]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear']  # Focus only on compatible penalties and solvers
}

grid = GridSearchCV(LogisticRegression(), param_grid, scoring='f1', cv=5)
grid.fit(X_train, y_train)
print("Best parameters:", grid.best_params_)


Best parameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
